In [2]:
import os, shutil
original_data_dir = '/home/shuvendu/Documents/datasets/BanglaLekha-Isolated/Images'
train_dir = '/home/shuvendu/Documents/datasets/BanglaLekha-Isolated/train'
test_dir = '/home/shuvendu/Documents/datasets/BanglaLekha-Isolated/test'

In [ ]:
os.mkdir(test_dir)
os.mkdir(train_dir)

In [ ]:
from os import listdir
from os.path import isfile, join

for i in range(1, 85):
    src_path = os.path.join(original_data_dir, str(i))
    train_path = os.path.join(train_dir, str(i))
    test_path = os.path.join(test_dir, str(i))
    os.mkdir(train_path)
    os.mkdir(test_path)
    
    onlyfiles = [f for f in listdir(src_path) if isfile(join(src_path, f))]
    for j in range(1600):
        src = os.path.join(src_path, onlyfiles[j])
        dst = os.path.join(train_path, onlyfiles[j])
        shutil.copyfile(src, dst)
    
    for j in range(1600, len(onlyfiles)):
        src = os.path.join(src_path, onlyfiles[j])
        dst = os.path.join(test_path, onlyfiles[j])
        shutil.copyfile(src, dst)


In [3]:
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.
/home/shuvendu/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=32
)

test_generator = train_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=32
)

Found 134400 images belonging to 84 classes.
Found 31705 images belonging to 84 classes.


In [6]:
for data_batch, label_batch in train_generator:
    print(data_batch.shape)
    print(label_batch.shape)
    break

(64, 150, 150, 3)
(64, 84)


In [11]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(84, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [12]:
# Compile the model
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy]
)

In [ ]:
history = model.fit_generator(train_generator, 
                              steps_per_epoch=4200, 
                              epochs=30, 
                              validation_data=test_generator, 
                              validation_steps=900)

Epoch 1/30
 605/4200 [===>..........................] - ETA: 1396s - loss: 2.5639 - categorical_accuracy: 0.3786